<a href="https://colab.research.google.com/github/Inteli-College/2024-2A-T05-CC09-G04/blob/tracking-SP05/enya_sp05_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opencv-python-headless scikit-image
!pip install git+https://github.com/ultralytics/yolov5  # YOLOv5 pode ser substituído por YOLOv8 conforme sua necessidade
!pip install filterpy  # filterpy contém implementações úteis para algoritmos de tracking como o SORT
!git clone https://github.com/abewley/sort.git
!cd sort && pip install


  Cloning https://github.com/ultralytics/yolov5 to /tmp/pip-req-build-pj7csrai
  Running command git clone --filter=blob:none --quiet https://github.com/ultralytics/yolov5 /tmp/pip-req-build-pj7csrai
  Resolved https://github.com/ultralytics/yolov5 to commit 6629839d5f51099c46a51dc2854901a4174cecc8
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
fatal: destination path 'sort' already exists and is not an empty directory.
ERROR: You must give at least one requirement to install (see "p

In [9]:
# Clonar o repositório do YOLOv5
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5

# Instalar as dependências
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 16977, done.
remote: Counting objects: 100% (172/172), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 16977 (delta 90), reused 109 (delta 53), pack-reused 16805 (from 1)
Receiving objects: 100% (16977/16977), 15.71 MiB | 31.85 MiB/s, done.
Resolving deltas: 100% (11619/11619), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.5/882.5 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.9 MB/s eta 0:00:00


In [2]:
!cat /content/sort/sort.py


"""
    SORT: A Simple, Online and Realtime Tracker
    Copyright (C) 2016-2020 Alex Bewley alex@bewley.ai

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.
"""
from __future__ import print_function

import os
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage import io

import glob
import time
import argp

In [3]:
!ls -l sort


total 64
drwxr-xr-x 3 root root  4096 Oct  9 02:16 data
-rw-r--r-- 1 root root 35141 Oct  9 02:16 LICENSE
-rw-r--r-- 1 root root  5416 Oct  9 02:16 README.md
-rw-r--r-- 1 root root    48 Oct  9 02:16 requirements.txt
-rw-r--r-- 1 root root 11737 Oct  9 02:26 sort.py


In [4]:
!sed -i "s/matplotlib.use('TkAgg')/matplotlib.use('Agg')/" /content/sort/sort.py


In [5]:
!grep "matplotlib.use" /content/sort/sort.py


matplotlib.use('Agg')


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import sys
sys.path.append('/content/sort')  # Ajuste este caminho conforme necessário
from sort import Sort


In [24]:
import cv2
import torch
import numpy as np
from sort import Sort
from ultralytics import YOLO
import os



In [27]:
model = YOLO('yolov8n')
dataset_path = '/content/drive/MyDrive/dataset/data.yaml'

model.train(
    data=os.path.join(dataset_path),  # Arquivo YAML com as classes e caminhos
    epochs=15,  # Definindo 15 épocas para acelerar o processo
    imgsz=640,  # Tamanho das imagens para treinamento
    batch=8,  # Tamanho do lote, ajustado para evitar problemas de memória
    name='projeto_so05',  # Nome do projeto de treinamento
    save=True  # Salvando checkpoints durante o treinamento
)


Ultralytics 8.3.8 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/dataset/data.yaml, epochs=15, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=projeto_so052, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_c

100%|██████████| 755k/755k [00:00<00:00, 13.9MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 66.1MB/s]


AMP: checks passed ✅


...

                   all        617        617      0.883       0.87      0.905      0.545



15 epochs completed in 0.162 hours.
Optimizer stripped from runs/detect/projeto_so052/weights/last.pt, 5.6MB
Optimizer stripped from runs/detect/projeto_so052/weights/best.pt, 5.6MB

Validating runs/detect/projeto_so052/weights/best.pt...
Ultralytics 8.3.8 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 186 layers, 2,684,563 parameters, 0 gradients, 6.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 39/39 [00:04<00:00,  9.21it/s]


                   all        617        617      0.883       0.87      0.905      0.545
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/projeto_so052


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d82ac2fd930>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [37]:
results = model(frame)
print(type(results))  # Para ver o tipo de objeto que é 'results'
print(results)        # Para ver a estrutura detalhada de 'results'



0: 384x640 1 face, 8.9ms
Speed: 2.1ms preprocess, 8.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
<class 'list'>
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'face'}
obb: None
orig_img: array([[[ 12,  24,  29],
        [ 12,  24,  29],
        [ 12,  24,  29],
        ...,
        [250, 253, 251],
        [250, 253, 251],
        [250, 253, 251]],

       [[ 12,  24,  29],
        [ 12,  24,  29],
        [ 12,  24,  29],
        ...,
        [250, 253, 251],
        [250, 253, 251],
        [250, 253, 251]],

       [[ 12,  24,  29],
        [ 12,  24,  29],
        [ 12,  24,  29],
        ...,
        [250, 253, 251],
        [250, 253, 251],
        [250, 253, 251]],

       ...,

       [[ 61,  93, 120],
        [ 61,  93, 120],
        [ 59,  91, 118],
        ...,
        [136, 150, 150],
        [136, 150, 150],
        [136, 150, 150]],

       [[ 60,  

In [43]:
import cv2
import numpy as np

# Carregar vídeo
video_path = '/content/drive/MyDrive/Colab_Notebooks/enya-sp04/video-novo-cortado.mp4'
cap = cv2.VideoCapture(video_path)

# Configurar saída de vídeo
output_video_path = '/content/drive/MyDrive/Colab_Notebooks/enya-sp04/tracked_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Detecção de objetos com YOLO
    results = model(frame)
    results = results[0] if isinstance(results, list) else results

    # Imprimir para verificar a estrutura dos resultados
    print("Tipo de 'results':", type(results))
    if hasattr(results, 'boxes'):
        print("Tipo de 'results.boxes':", type(results.boxes))
        print("Conteúdo de 'results.boxes':", results.boxes)

    # Extrair informações das caixas de detecção, se possível
    detections = []
    if hasattr(results, 'pred') and results.pred is not None:  # Ajustar 'pred' conforme apropriado
        for pred in results.pred:
            x1, y1, x2, y2, confidence, class_id = pred
            if confidence > 0.3:
                detections.append([x1, y1, x2, y2, confidence])

    # Atualizando o rastreador SORT com detecções
    detections_np = np.array(detections)
    if len(detections_np) > 0:
        tracked_objects = tracker.update(detections_np)
        for x1, y1, x2, y2, obj_id in tracked_objects:
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(frame, f'ID {int(obj_id)}', (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Salvar o frame no vídeo de saída
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()


A saída de streaming foi truncada nas últimas 5000 linhas.
Tipo de 'results.boxes': <class 'ultralytics.engine.results.Boxes'>
Conteúdo de 'results.boxes': ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.], device='cuda:0')
conf: tensor([0.36543], device='cuda:0')
data: tensor([[5.02438e+01, 3.88081e+01, 1.26108e+03, 7.20000e+02, 3.65435e-01, 0.00000e+00]], device='cuda:0')
id: None
is_track: False
orig_shape: (720, 1280)
shape: torch.Size([1, 6])
xywh: tensor([[ 655.66217,  379.40405, 1210.83679,  681.19189]], device='cuda:0')
xywhn: tensor([[0.51224, 0.52695, 0.94597, 0.94610]], device='cuda:0')
xyxy: tensor([[  50.24377,   38.80811, 1261.08057,  720.00000]], device='cuda:0')
xyxyn: tensor([[0.03925, 0.05390, 0.98522, 1.00000]], device='cuda:0')

0: 384x640 1 face, 8.2ms
Speed: 1.9ms preprocess, 8.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
Tipo de 'results': <class 'ultralytics.engine.results.Results'>
Tipo de 'results.boxes': <class